###  Long Short-Term Memory model (see Section 3.2. of thesis)

This notebook focuses on training and testing the  Long Short-Term Memory  that were proposed in this paper. The model was implemented using TensorFlows.

Please keep in mind that these notebooks are primarily used for conducting experiments, live coding, and implementing and evaluating the approaches presented in the thesis. As a result, the code in this notebook may not strictly adhere to best practice coding standards.

In [ ]:
# only execute once
# ONLY IF USED ON LOCAL VIEW
import os

# Getting the parent directory
os.chdir("..")
os.chdir("..")

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences


def import_test_train(local):
  """
  This imports the given train and testset locally or not and returns it.

  :param local: If set to true, it will return the trainset from a local view. Otherwise it will open drive mount and attempts to connect to your
  drive folders.
  """

  assert type(local) == bool, f"Type is not valid. Expected boolean, recieved: {type(local)}"

  if local:
    from google.colab import drive
    drive.mount('/content/gdrive')

    df_test = pd.read_csv('/content/gdrive/MyDrive/Experiment/testset_DE_Trigger.csv')
    df_train = pd.read_csv('/content/gdrive/MyDrive/Experiment/trainset_DE_Trigger.csv')

    return df_test, df_train

  else:
    df_test = pd.read_csv('./Experiment/testset_DE_Trigger.csv')
    df_train = pd.read_csv('./Experiment/trainset_DE_Trigger.csv')

    return df_test, df_train

# importing test and trainset
df_test, df_train = import_test_train(True)

# If you want to use it locally, make sure to execute the notebooks from the root directory of this project and uncomment the following line:
# df_test, df_train = import_test_train(False)

Mounted at /content/gdrive


## Define Labels as numbers

In [4]:
MAX_NB_WORDS = 39000
MAX_SEQUENCE_LENGTH = 150
HIDDEN_DIM = 200

tokenizer = Tokenizer(num_words=MAX_NB_WORDS, lower=True)
tokenizer.fit_on_texts(df_test.append(df_train)["content"].values)
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

<ipython-input-4-493ffa27cf9c>:6: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  tokenizer.fit_on_texts(df_test.append(df_train)["content"].values)


Found 44770 unique tokens.


In [5]:
X_train = tokenizer.texts_to_sequences(df_train['content'].values)
X_train = pad_sequences(X_train, maxlen=MAX_SEQUENCE_LENGTH)

Y_train = pd.get_dummies(df_train['label']).values

X_test = tokenizer.texts_to_sequences(df_test['content'].values)
X_test = pad_sequences(X_test, maxlen=MAX_SEQUENCE_LENGTH)

Y_test = pd.get_dummies(df_test['label']).values


## Define model and saving path

In [6]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, BatchNormalization, Dropout

def emotion_model():
    model = Sequential()
    model.add(Embedding(MAX_NB_WORDS, HIDDEN_DIM, input_length=X_train.shape[1]))
    model.add(LSTM(100, dropout=0.2, recurrent_dropout=0.2))
    model.add(BatchNormalization())
    model.add(Dense(256,activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(5, activation='softmax'))

    model.compile(loss=tf.keras.losses.CategoricalCrossentropy(),
              optimizer='adam',metrics=[tf.keras.metrics.AUC()])

    return model

## Train model

In [7]:
from tensorflow.keras.callbacks import EarlyStopping

# Set the early stopping criteria
early_stopping = EarlyStopping(monitor='val_loss', patience=2)

# Create the model
model = emotion_model()

epochs = 100
batch_size = 64

# Fit the model with early stopping
model.fit(
    X_train, Y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_split=0.1,
    callbacks=[early_stopping]
)

### Evaluation

In [7]:
from sklearn.metrics import classification_report

y_labels = [list(i).index(1) for i in Y_test]
Y_pred = np.argmax(model.predict(X_test),axis=1)

print(classification_report(y_labels, Y_pred))